In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

RANDOM_STATE = 42
TEST_SIZE = 0.15
VAL_SIZE = 0.15 / (1 - TEST_SIZE)
N_CV_FOLDS = 10

BANDGAP_FEATURES = [
    'B1_electronegativity', 'B2_electronegativity', 'A_atomic_mass', 'B1_atomic_mass',
    'X_electron_affinity', 'B2_atomic_number', 'B1_first_ionization_energy',
    'B2_first_ionization_energy', 'X_first_ionization_energy', 'A_second_ionization_energy',
    'B1_second_ionization_energy', 'B1_melting_point', 'B2_melting_point',
    'B1_boiling_point', 'B1_third_ionization_energy', 'B2_third_ionization_energy'
]

FORMATION_FEATURES = [
    'A_electronegativity', 'B1_electronegativity', 'B2_electronegativity',
    'X_electron_affinity', 'X_first_ionization_energy', 'A_second_ionization_energy'
]

class BandgapPredictor:
    
    def __init__(self, random_state=RANDOM_STATE):
        self.random_state = random_state
        self.scaler = MinMaxScaler()
        self.gbr_bandgap = None
        self.gbr_formation = None
        self.X_train_bg = None
        self.X_val_bg = None
        self.X_test_bg = None
        self.y_train_bg = None
        self.y_val_bg = None
        self.y_test_bg = None
        self.X_train_fe = None
        self.X_val_fe = None
        self.X_test_fe = None
        self.y_train_fe = None
        self.y_val_fe = None
        self.y_test_fe = None
        
        self.results = {
            'bandgap': {},
            'formation_energy': {}
        }
    
    def load_data(self, filepath):
        self.df = pd.read_excel(filepath)
        print(f"Dataset loaded: {self.df.shape[0]} samples, {self.df.shape[1]} features")
        return self.df
    
    def preprocess_data(self):
        print("DATA PREPROCESSING")
        
        feature_cols = [col for col in self.df.columns 
                       if col not in ['A2BBX6', 'A', 'B1', 'B2', 'X', 
                                     'band_gap', 'formation_energy']]
        
        X = self.df[feature_cols].copy()
        self.y_bandgap = self.df['band_gap'].copy()
        self.y_formation = self.df['formation_energy'].copy()
        
        X_normalized = self.scaler.fit_transform(X)
        self.X_normalized = pd.DataFrame(X_normalized, columns=feature_cols)
        
        print(f"\nDataset shape: {self.X_normalized.shape}")
        print(f"Features normalized to range: [{self.X_normalized.min().min():.4f}, "
              f"{self.X_normalized.max().max():.4f}]")
        print(f"\nBandgap - Min: {self.y_bandgap.min():.4f}, "
              f"Max: {self.y_bandgap.max():.4f}, Mean: {self.y_bandgap.mean():.4f}")
        print(f"Formation Energy - Min: {self.y_formation.min():.4f}, "
              f"Max: {self.y_formation.max():.4f}, Mean: {self.y_formation.mean():.4f}")
    
    def select_features(self):
        print("FEATURE SELECTION")
        
        bandgap_features = [f for f in BANDGAP_FEATURES 
                           if f in self.X_normalized.columns]
        formation_features = [f for f in FORMATION_FEATURES 
                             if f in self.X_normalized.columns]
        
        self.X_bandgap = self.X_normalized[bandgap_features]
        self.X_formation = self.X_normalized[formation_features]
        
        print(f"\nBandgap prediction features ({len(bandgap_features)}):")
        for i, f in enumerate(bandgap_features, 1):
            print(f"  {i:2d}. {f}")
        
        print(f"\nFormation energy prediction features ({len(formation_features)}):")
        for i, f in enumerate(formation_features, 1):
            print(f"  {i:2d}. {f}")
        
        print(f"\nX_bandgap shape: {self.X_bandgap.shape}")
        print(f"X_formation shape: {self.X_formation.shape}")
    
    def split_data(self):
        print("DATA SPLITTING")
        
        self.X_train_bg, self.X_test_bg, self.y_train_bg, self.y_test_bg = \
            train_test_split(self.X_bandgap, self.y_bandgap, 
                           test_size=TEST_SIZE, random_state=self.random_state)
        
        self.X_train_bg, self.X_val_bg, self.y_train_bg, self.y_val_bg = \
            train_test_split(self.X_train_bg, self.y_train_bg, 
                           test_size=VAL_SIZE, random_state=self.random_state)
        
        self.X_train_fe, self.X_test_fe, self.y_train_fe, self.y_test_fe = \
            train_test_split(self.X_formation, self.y_formation, 
                           test_size=TEST_SIZE, random_state=self.random_state)
        
        self.X_train_fe, self.X_val_fe, self.y_train_fe, self.y_val_fe = \
            train_test_split(self.X_train_fe, self.y_train_fe, 
                           test_size=VAL_SIZE, random_state=self.random_state)
        
        print(f"\nBandgap Data:")
        print(f"  Training set: {self.X_train_bg.shape[0]} samples (68.8%)")
        print(f"  Validation set: {self.X_val_bg.shape[0]} samples (15.0%)")
        print(f"  Test set: {self.X_test_bg.shape[0]} samples (15.0%)")
        print(f"  Total: {self.X_train_bg.shape[0] + self.X_val_bg.shape[0] + self.X_test_bg.shape[0]} samples")
        
        print(f"\nFormation Energy Data:")
        print(f"  Training set: {self.X_train_fe.shape[0]} samples (68.8%)")
        print(f"  Validation set: {self.X_val_fe.shape[0]} samples (15.0%)")
        print(f"  Test set: {self.X_test_fe.shape[0]} samples (15.0%)")
        print(f"  Total: {self.X_train_fe.shape[0] + self.X_val_fe.shape[0] + self.X_test_fe.shape[0]} samples")
    
    def train_models(self):
        print("TRAINING GRADIENT BOOSTING REGRESSION MODELS")
        
        print("Gradient Boosting Regressor - BANDGAP PREDICTION")
        
        self.gbr_bandgap = GradientBoostingRegressor(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=3,
            min_samples_split=2,
            min_samples_leaf=1,
            subsample=1.0,
            random_state=self.random_state,
            verbose=0
        )
        
        self.gbr_bandgap.fit(self.X_train_bg, self.y_train_bg)
        self._evaluate_model(self.gbr_bandgap, self.X_train_bg, self.X_val_bg, 
                            self.X_test_bg, self.y_train_bg, self.y_val_bg, 
                            self.y_test_bg, 'bandgap')
        
        print("Gradient Boosting Regressor - FORMATION ENERGY PREDICTION")
        
        self.gbr_formation = GradientBoostingRegressor(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=3,
            min_samples_split=2,
            min_samples_leaf=1,
            subsample=1.0,
            random_state=self.random_state,
            verbose=0
        )
        
        self.gbr_formation.fit(self.X_train_fe, self.y_train_fe)
        self._evaluate_model(self.gbr_formation, self.X_train_fe, self.X_val_fe, 
                            self.X_test_fe, self.y_train_fe, self.y_val_fe, 
                            self.y_test_fe, 'formation_energy')
    
    def _evaluate_model(self, model, X_train, X_val, X_test, 
                       y_train, y_val, y_test, target_name):
        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)
        y_test_pred = model.predict(X_test)
        
        train_r2 = r2_score(y_train, y_train_pred)
        val_r2 = r2_score(y_val, y_val_pred)
        test_r2 = r2_score(y_test, y_test_pred)
        
        train_mse = mean_squared_error(y_train, y_train_pred)
        val_mse = mean_squared_error(y_val, y_val_pred)
        test_mse = mean_squared_error(y_test, y_test_pred)
        
        train_mae = mean_absolute_error(y_train, y_train_pred)
        val_mae = mean_absolute_error(y_val, y_val_pred)
        test_mae = mean_absolute_error(y_test, y_test_pred)
        
        test_rmse = np.sqrt(test_mse)
        
        self.results[target_name] = {
            'train': {'r2': train_r2, 'mse': train_mse, 'mae': train_mae},
            'val': {'r2': val_r2, 'mse': val_mse, 'mae': val_mae},
            'test': {'r2': test_r2, 'mse': test_mse, 'mae': test_mae, 'rmse': test_rmse},
            'model': model
        }
        
        print(f"\nTraining Set:")
        print(f"  R squared: {train_r2:.4f}")
        print(f"  MSE: {train_mse:.4f}")
        print(f"  MAE: {train_mae:.4f}")
        
        print(f"\nValidation Set:")
        print(f"  R squared: {val_r2:.4f}")
        print(f"  MSE: {val_mse:.4f}")
        print(f"  MAE: {val_mae:.4f}")
        
        print(f"\nTest Set:")
        print(f"  R squared: {test_r2:.4f}")
        print(f"  MSE: {test_mse:.4f}")
        print(f"  MAE: {test_mae:.4f}")
        print(f"  RMSE: {test_rmse:.4f}")
    
    def feature_importance(self):
        print("FEATURE IMPORTANCE ANALYSIS")
        
        feature_importance_bg = pd.DataFrame({
            'Feature': self.X_bandgap.columns,
            'Importance': self.gbr_bandgap.feature_importances_
        }).sort_values('Importance', ascending=False)
        
        print("\nBandgap - Feature Importance Ranking:")
        print(feature_importance_bg.to_string(index=False))
        
        feature_importance_fe = pd.DataFrame({
            'Feature': self.X_formation.columns,
            'Importance': self.gbr_formation.feature_importances_
        }).sort_values('Importance', ascending=False)
        
        print("\n\nFormation Energy - Feature Importance Ranking:")
        print(feature_importance_fe.to_string(index=False))
        
        return feature_importance_bg, feature_importance_fe
    
    def cross_validation(self):
        print("10-FOLD CROSS-VALIDATION")
        
        kf = KFold(n_splits=N_CV_FOLDS, shuffle=True, random_state=self.random_state)
        
        cv_scores_bg = []
        for fold, (train_idx, val_idx) in enumerate(kf.split(self.X_bandgap), 1):
            X_train_cv = self.X_bandgap.iloc[train_idx]
            y_train_cv = self.y_bandgap.iloc[train_idx]
            X_val_cv = self.X_bandgap.iloc[val_idx]
            y_val_cv = self.y_bandgap.iloc[val_idx]
            
            model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,
                                             max_depth=3, random_state=self.random_state)
            model.fit(X_train_cv, y_train_cv)
            y_pred_cv = model.predict(X_val_cv)
            r2 = r2_score(y_val_cv, y_pred_cv)
            cv_scores_bg.append(r2)
        
        print("\nBandgap - 10-Fold Cross-Validation R squared Scores:")
        for i, score in enumerate(cv_scores_bg, 1):
            print(f"  Fold {i:2d}: {score:.4f}")
        print(f"  Mean: {np.mean(cv_scores_bg):.4f} (plus-minus {np.std(cv_scores_bg):.4f})")
        
        cv_scores_fe = []
        for fold, (train_idx, val_idx) in enumerate(kf.split(self.X_formation), 1):
            X_train_cv = self.X_formation.iloc[train_idx]
            y_train_cv = self.y_formation.iloc[train_idx]
            X_val_cv = self.X_formation.iloc[val_idx]
            y_val_cv = self.y_formation.iloc[val_idx]
            
            model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,
                                             max_depth=3, random_state=self.random_state)
            model.fit(X_train_cv, y_train_cv)
            y_pred_cv = model.predict(X_val_cv)
            r2 = r2_score(y_val_cv, y_pred_cv)
            cv_scores_fe.append(r2)
        
        print("\n\nFormation Energy - 10-Fold Cross-Validation R squared Scores:")
        for i, score in enumerate(cv_scores_fe, 1):
            print(f"  Fold {i:2d}: {score:.4f}")
        print(f"  Mean: {np.mean(cv_scores_fe):.4f} (plus-minus {np.std(cv_scores_fe):.4f})")
        
        return cv_scores_bg, cv_scores_fe
    
    def compare_models(self):
        print("MODEL COMPARISON")
        
        results_comparison = []
        
        print("\nBandgap Prediction - Algorithm Comparison:")
        
        for name, model_class in [('GBR', GradientBoostingRegressor), 
                                  ('RF', RandomForestRegressor)]:
            if name == 'GBR':
                model = model_class(n_estimators=100, learning_rate=0.1, 
                                   max_depth=3, random_state=self.random_state)
            else:
                model = model_class(n_estimators=100, max_depth=10, 
                                   random_state=self.random_state)
            
            model.fit(self.X_train_bg, self.y_train_bg)
            y_pred = model.predict(self.X_test_bg)
            
            r2 = r2_score(self.y_test_bg, y_pred)
            mse = mean_squared_error(self.y_test_bg, y_pred)
            mae = mean_absolute_error(self.y_test_bg, y_pred)
            rmse = np.sqrt(mse)
            
            results_comparison.append({
                'Target': 'Bandgap',
                'Model': name,
                'R_squared': r2,
                'MSE': mse,
                'MAE': mae,
                'RMSE': rmse
            })
            
            print(f"\n{name}:")
            print(f"  R squared: {r2:.4f}, MSE: {mse:.4f}, MAE: {mae:.4f}, RMSE: {rmse:.4f}")
        
        print("\n\nFormation Energy Prediction - Algorithm Comparison:")
        
        for name, model_class in [('GBR', GradientBoostingRegressor), 
                                  ('RF', RandomForestRegressor)]:
            if name == 'GBR':
                model = model_class(n_estimators=100, learning_rate=0.1, 
                                   max_depth=3, random_state=self.random_state)
            else:
                model = model_class(n_estimators=100, max_depth=10, 
                                   random_state=self.random_state)
            
            model.fit(self.X_train_fe, self.y_train_fe)
            y_pred = model.predict(self.X_test_fe)
            
            r2 = r2_score(self.y_test_fe, y_pred)
            mse = mean_squared_error(self.y_test_fe, y_pred)
            mae = mean_absolute_error(self.y_test_fe, y_pred)
            rmse = np.sqrt(mse)
            
            results_comparison.append({
                'Target': 'Formation Energy',
                'Model': name,
                'R_squared': r2,
                'MSE': mse,
                'MAE': mae,
                'RMSE': rmse
            })
            
            print(f"\n{name}:")
            print(f"  R squared: {r2:.4f}, MSE: {mse:.4f}, MAE: {mae:.4f}, RMSE: {rmse:.4f}")
        
        results_df = pd.DataFrame(results_comparison)
        print("\n\nComparison Summary Table:")
        print(results_df.to_string(index=False))
        
        return results_df
    
    def plot_predictions(self):
        
        print("GENERATING PLOTS")
        
        y_train_pred_bg = self.gbr_bandgap.predict(self.X_train_bg)
        y_val_pred_bg = self.gbr_bandgap.predict(self.X_val_bg)
        y_test_pred_bg = self.gbr_bandgap.predict(self.X_test_bg)
        
        fig, ax = plt.subplots(figsize=(12, 9))
        ax.scatter(self.y_train_bg, y_train_pred_bg, alpha=0.5, s=40, 
                  label='Training Data', color='green', edgecolors='darkgreen', linewidth=0.5)
        ax.scatter(self.y_val_bg, y_val_pred_bg, alpha=0.5, s=40, 
                  label='Validation Data', color='orange', edgecolors='darkorange', linewidth=0.5)
        ax.scatter(self.y_test_bg, y_test_pred_bg, alpha=0.5, s=40, 
                  label='Test Data', color='red', edgecolors='darkred', linewidth=0.5)
        
        min_val = min(self.y_train_bg.min(), self.y_val_bg.min(), self.y_test_bg.min(),
                     y_train_pred_bg.min(), y_val_pred_bg.min(), y_test_pred_bg.min())
        max_val = max(self.y_train_bg.max(), self.y_val_bg.max(), self.y_test_bg.max(),
                     y_train_pred_bg.max(), y_val_pred_bg.max(), y_test_pred_bg.max())
        ax.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2.5, label='Perfect Prediction')
        
        r2 = self.results['bandgap']['test']['r2']
        mae = self.results['bandgap']['test']['mae']
        rmse = self.results['bandgap']['test']['rmse']
        
        ax.set_xlabel('Actual Bandgap (eV)', fontsize=12, fontweight='bold')
        ax.set_ylabel('Predicted Bandgap (eV)', fontsize=12, fontweight='bold')
        ax.set_title('Bandgap Prediction: Predicted vs Actual', fontsize=14, fontweight='bold')
        
        textstr = f'Test R squared: {r2:.4f}\nTest MAE: {mae:.4f} eV\nTest RMSE: {rmse:.4f} eV'
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.8)
        ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=11,
                verticalalignment='top', bbox=props, family='monospace')
        
        ax.grid(True, alpha=0.3)
        ax.legend(loc='lower right', fontsize=10)
        plt.tight_layout()
        plt.show()
        print("Bandgap prediction plot displayed")
        
        y_train_pred_fe = self.gbr_formation.predict(self.X_train_fe)
        y_val_pred_fe = self.gbr_formation.predict(self.X_val_fe)
        y_test_pred_fe = self.gbr_formation.predict(self.X_test_fe)
        
        fig, ax = plt.subplots(figsize=(12, 9))
        ax.scatter(self.y_train_fe, y_train_pred_fe, alpha=0.5, s=40, 
                  label='Training Data', color='green', edgecolors='darkgreen', linewidth=0.5)
        ax.scatter(self.y_val_fe, y_val_pred_fe, alpha=0.5, s=40, 
                  label='Validation Data', color='orange', edgecolors='darkorange', linewidth=0.5)
        ax.scatter(self.y_test_fe, y_test_pred_fe, alpha=0.5, s=40, 
                  label='Test Data', color='red', edgecolors='darkred', linewidth=0.5)
        
        min_val = min(self.y_train_fe.min(), self.y_val_fe.min(), self.y_test_fe.min(),
                     y_train_pred_fe.min(), y_val_pred_fe.min(), y_test_pred_fe.min())
        max_val = max(self.y_train_fe.max(), self.y_val_fe.max(), self.y_test_fe.max(),
                     y_train_pred_fe.max(), y_val_pred_fe.max(), y_test_pred_fe.max())
        ax.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2.5, label='Perfect Prediction')
        
        r2 = self.results['formation_energy']['test']['r2']
        mae = self.results['formation_energy']['test']['mae']
        rmse = self.results['formation_energy']['test']['rmse']
        
        ax.set_xlabel('Actual Formation Energy (eV/atom)', fontsize=12, fontweight='bold')
        ax.set_ylabel('Predicted Formation Energy (eV/atom)', fontsize=12, fontweight='bold')
        ax.set_title('Formation Energy Prediction: Predicted vs Actual', fontsize=14, fontweight='bold')
        
        textstr = f'Test R squared: {r2:.4f}\nTest MAE: {mae:.4f} eV/atom\nTest RMSE: {rmse:.4f} eV/atom'
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.8)
        ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=11,
                verticalalignment='top', bbox=props, family='monospace')
        
        ax.grid(True, alpha=0.3)
        ax.legend(loc='lower right', fontsize=10)
        plt.tight_layout()
        plt.show()
        print("Formation energy prediction plot displayed")
        
    def run_full_pipeline(self, filepath):
        print("GRADIENT BOOSTING MODEL FOR BANDGAP PREDICTION")
        print("Lead-Free Double Perovskite Photovoltaic Materials")
        
        self.load_data(filepath)
        self.preprocess_data()
        self.select_features()
        self.split_data()
        self.train_models()
        self.cross_validation()
        self.compare_models()
        
        self.plot_predictions()
        

if __name__ == "__main__":
    predictor = BandgapPredictor(random_state=RANDOM_STATE)
    
    predictor.run_full_pipeline('dataset_1053.xlsx')

GRADIENT BOOSTING MODEL FOR BANDGAP PREDICTION
Lead-Free Double Perovskite Photovoltaic Materials


FileNotFoundError: [Errno 2] No such file or directory: 'dataset_1053.xlsx'